In [1]:
import simpy
import random

In [2]:
SIMULATION_TIME = 24 * 60
VESSEL_AVG_ARRIVAL_INTERVAL = 5 * 60
CONTAINERS_PER_VESSEL = 150
AVAIL_BERTHS = 2
AVAIL_CRANES = 2
AVAIL_TRUCKS = 3
MOVE_CONTAINER_TIME = 3
TRUCK_TRIP_ROUND_TIME = 6

In [3]:
class EV:
    def __init__(self, env, berths, cranes, trucks):
        self.env = env
        self.berths = simpy.Resource(env, berths)
        self.cranes = simpy.Resource(env, cranes)
        self.trucks = simpy.Resource(env, trucks)


    def handle_vessel(self, vessel_id):
        print(f"Vessel {vessel_id} arriving at time {self.env.now}")

        with self.berths.request() as req:
            yield req
            print(f"Vessel {vessel_id} berthing at time {self.env.now}")
            
            for container_no in range(CONTAINERS_PER_VESSEL):
                with self.cranes.request() as crane_req:
                    yield crane_req
                    
                    yield self.env.process(self.move_container(vessel_id, container_no+1))
            
            print(f"Vessel {vessel_id} leaving at time {self.env.now}")


    def move_container(self, vessel_id, container_no):
        with self.trucks.request() as req:
            yield req
    
            print(f"Quay crane moving container {container_no} from vessel {vessel_id} at time {self.env.now}")
            yield self.env.timeout(MOVE_CONTAINER_TIME)
            
            print(f"Truck transporting container {container_no} from vessel {vessel_id} at time {self.env.now}")
            yield self.env.timeout(TRUCK_TRIP_ROUND_TIME)
            
            print(f"Truck returned after transporting container {container_no} from vessel {vessel_id} at time {self.env.now}")
    

In [4]:
def vessel_generator(env):
    vessel = EV(env, AVAIL_BERTHS, AVAIL_CRANES, AVAIL_TRUCKS)
    
    vessel_id = 0
    while(True):
        yield env.timeout(random.expovariate(1 / VESSEL_AVG_ARRIVAL_INTERVAL))
        vessel_id += 1
        env.process(vessel.handle_vessel(vessel_id))

In [5]:
env = simpy.Environment()
env.process(vessel_generator(env))
env.run(SIMULATION_TIME)

Vessel 1 arriving at time 309.9345608057552
Vessel 1 berthing at time 309.9345608057552
Quay crane moving container 1 from vessel 1 at time 309.9345608057552
Truck transporting container 1 from vessel 1 at time 312.9345608057552
Truck returned after transporting container 1 from vessel 1 at time 318.9345608057552
Quay crane moving container 2 from vessel 1 at time 318.9345608057552
Truck transporting container 2 from vessel 1 at time 321.9345608057552
Truck returned after transporting container 2 from vessel 1 at time 327.9345608057552
Quay crane moving container 3 from vessel 1 at time 327.9345608057552
Truck transporting container 3 from vessel 1 at time 330.9345608057552
Truck returned after transporting container 3 from vessel 1 at time 336.9345608057552
Quay crane moving container 4 from vessel 1 at time 336.9345608057552
Truck transporting container 4 from vessel 1 at time 339.9345608057552
Truck returned after transporting container 4 from vessel 1 at time 345.9345608057552
Quay